In [21]:
import psycopg2
import pandas as pd
import numpy as np
import mariadb
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
import pickle
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import seaborn as sns

In [38]:
os.getcwd()

'D:\\Cropnuts\\DSML158\\SoilAnalysis'

In [52]:
soil_df = pd.read_csv("output/soil_analysis_cleaned.csv")

In [53]:
soil_df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,7,7,3,3,CF045SA0784,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,61.0,uS/cm
1,58,58,29,29,CF045SA0786,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,240.0,uS/cm
2,90,90,45,45,CF045SA0787,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,30.0,uS/cm
3,122,122,61,61,CF045SA0788,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,119.0,uS/cm
4,177,177,95,95,CF045SA0797,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,39.0,uS/cm
...,...,...,...,...,...,...,...,...,...,...
27801,1104,1104,696,696,CW015SA0046,2012-03-05 00:00:00.000,Heavy Metals Analysis,aluminium,639.0,ppm
27802,1189,1189,713,713,CW015SA0047,2012-03-05 00:00:00.000,Heavy Metals Analysis,aluminium,254.0,ppm
27803,1239,1239,723,723,CW015SA0048,2012-03-05 00:00:00.000,Heavy Metals Analysis,aluminium,923.0,ppm
27804,13717,13717,7126,7126,CW015SA0043,2012-02-29 00:00:00.000,Heavy Metals Analysis,aluminium,1395.0,ppm


In [54]:
new_samples_df = soil_df.loc[soil_df['batch_date'] > '2024-05-21'] 
soil_df = soil_df.loc[soil_df['batch_date'] <= '2024-05-21'] 

In [55]:
soil_df['chemical_name'] = [ i.lower().replace(" ","_").replace(".","").replace("(","").replace(")","").replace("/","").strip() for i in soil_df['chemical_name'] ]

In [56]:
for analysis in np.unique(soil_df['analysis_name']):
    print(analysis)
    df_ = soil_df.loc[soil_df['analysis_name']==analysis]
    df_ = pd.pivot_table(data=df_, values="result", index="sample_code", columns="chemical_name")
    print(len(df_))

    os.makedirs(f"output/boxplots/{analysis}",exist_ok=True)
    for column in df_.columns:
        # print(column)
        os.makedirs(f"output/chemical_null_count",exist_ok=True)
        plt.boxplot(df_[column])
        plt.savefig(f"output/boxplots/{analysis}/{column}.png")
        plt.clf()
        outlier_threshold = df_[column].quantile(0.99)
        # df_ = df_.loc[df_[column] <= outlier_threshold]
    df_.describe().to_csv(f"output/chemical_null_count/{analysis}.csv")
    print(len(df_))
    if(len(df_) == 0):
        continue
    os.makedirs(f"output/analysis",exist_ok=True)
    df_.to_csv(f"output/analysis/{analysis}.csv")
    

% Organic Matter (OM)
529
529
% Soil Nitrogen (N)
299
299
AFSIS Std Wet Chemistry Soil Analysis
185
185
Basic Soil Analysis (BSA)
23
23
Carbon Analysis
54
54
Complete Soil Analysis
866
866
Complete Soil Analysis (M3)
6
6
Complete Soil Analysis with Recommendations
65
65
Exch. Aluminium
66
66
Exchangeable Acidity (Hp) Analysis
28
28
Heavy Metals Analysis
6
6
Olsen P Analysis
55
55
SOIL_Scan
15
15
Soil Bulk Density Analysis
22
22
Soil Health Care Analysis
69
69
Soil Texture Analysis
63
63
Standard Soil (Olsen P)
243
243


<Figure size 640x480 with 0 Axes>

In [57]:
# for analysis_file in os.listdir("output/analysis"):
#     os.makedirs("./output/pairplots",exist_ok=True)
#     analysis_df = pd.read_csv(f"output/analysis/{analysis_file}",index_col=0)
#     print(analysis_file.replace('.csv',''))
#     sns.pairplot(analysis_df)
#     print('Saving')
#     plt.savefig(f"output/pairplots/{analysis_file.replace('.csv','')}.png")
#     print('Saved')
#     plt.clf()

In [58]:
for analysis_file in os.listdir("output/analysis"):
    print(analysis_file)
    analysis_df = pd.read_csv(f"output/analysis/{analysis_file}",index_col=0)
    print(len(analysis_df.columns))
    os.makedirs(f"models/imputers",exist_ok=True)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    analysis_df = pd.DataFrame(imp_mean.fit_transform(analysis_df), columns=analysis_df.columns, index=analysis_df.index)
    pickle.dump(imp_mean, open(f"models/imputers/{analysis_file.replace('csv','pkl')}","wb"))
    # if(len(analysis_df) == 0):
    #     continue
    sc = StandardScaler()
    pca = PCA()
    analysis_scaled = sc.fit_transform(analysis_df)
    pca.fit(analysis_scaled)
    pca_explained_variance_df = pd.DataFrame(zip(analysis_df.columns,pca.explained_variance_ratio_))
    os.makedirs(f"output/pca_explained_variance_ratio",exist_ok=True)
    pca_explained_variance_df.to_csv(f"output/pca_explained_variance_ratio/{analysis_file}")
    pca_explained_variance_df = pca_explained_variance_df.loc[pca_explained_variance_df[1]>0.1]
    n_components = len(pca_explained_variance_df)
    
    pca_chems = pca_explained_variance_df[0]
    
    pca = PCA(n_components=n_components)
    analysis_scaled = sc.fit_transform(analysis_df)
    pca_reduced_df = pca.fit_transform(analysis_scaled)
    analysis_scaled = pd.DataFrame(analysis_scaled,index=analysis_df.index)
    pca_reduced_df = pd.DataFrame(pca_reduced_df,index=analysis_df.index, columns=pca_chems)
    os.makedirs(f"output/analysis_scaled",exist_ok=True)
    os.makedirs(f"output/pca_df",exist_ok=True)
    os.makedirs(f"models/scalers",exist_ok=True)
    os.makedirs(f"models/pca",exist_ok=True)
    analysis_scaled.to_csv(f"output/analysis_scaled/{analysis_file}")
    pca_reduced_df.to_csv(f"output/pca_df/{analysis_file}")
    pickle.dump(sc, open(f"models/scalers/{analysis_file.replace('csv','pkl')}","wb"))
    pickle.dump(pca, open(f"models/pca/{analysis_file.replace('csv','pkl')}","wb"))

% Organic Matter (OM).csv
18
% Soil Nitrogen (N).csv
18
AFSIS Std Wet Chemistry Soil Analysis.csv
1
Basic Soil Analysis (BSA).csv
1
Carbon Analysis.csv
2
Complete Soil Analysis (M3).csv
1
Complete Soil Analysis with Recommendations.csv
3
Complete Soil Analysis.csv
18
Exch. Aluminium.csv
1
Exchangeable Acidity (Hp) Analysis.csv
2
Heavy Metals Analysis.csv
16
Olsen P Analysis.csv
16
Soil Bulk Density Analysis.csv
2
Soil Health Care Analysis.csv
1
Soil Texture Analysis.csv
1
SOIL_Scan.csv
1
Standard Soil (Olsen P).csv
1


In [59]:
for analysis_file in os.listdir("output/pca_df"):
    print(analysis_file)
    os.makedirs(f"output/mahalanobis_distance",exist_ok=True)
    pca_reduced_df = pd.read_csv(f"output/pca_df/{analysis_file}",index_col=0)
    # if len(pca_reduced_df.columns) < 2:
        # continue
    mu = np.mean(pca_reduced_df, axis=0)
    sigma = np.cov(pca_reduced_df.T)
    os.makedirs(f"output/pca_df",exist_ok=True)
    try:
        pca_reduced_df['mahalanobis_distance'] = [distance.mahalanobis(pca_reduced_df.iloc[i], mu, np.linalg.inv(sigma)) for i in range(len(pca_reduced_df)) ]
    except:
        continue
    pca_reduced_df.to_csv(f"output/mahalanobis_distance/{analysis_file}")

% Organic Matter (OM).csv
% Soil Nitrogen (N).csv
AFSIS Std Wet Chemistry Soil Analysis.csv
Basic Soil Analysis (BSA).csv
Carbon Analysis.csv
Complete Soil Analysis (M3).csv
Complete Soil Analysis with Recommendations.csv
Complete Soil Analysis.csv
Exch. Aluminium.csv
Exchangeable Acidity (Hp) Analysis.csv
Heavy Metals Analysis.csv
Olsen P Analysis.csv
Soil Bulk Density Analysis.csv
Soil Health Care Analysis.csv
Soil Texture Analysis.csv
SOIL_Scan.csv
Standard Soil (Olsen P).csv


In [60]:
mahalanobis_threshold_dict = {}
for analysis_file in os.listdir("output/mahalanobis_distance"):
    print(analysis_file)
    os.makedirs(f"output/mahalanobis_distance_upper_quantile",exist_ok=True)
    os.makedirs(f"output/mahalanobis_boxplots",exist_ok=True)
    analysis = analysis_file.replace(".csv","")
    mahalanobis_df = pd.read_csv(f"output/mahalanobis_distance/{analysis_file}",index_col=0)
    upper_quantile = (mahalanobis_df['mahalanobis_distance'].quantile(0.95))
    mahalanobis_threshold_dict[analysis_file.replace(".csv","")] = upper_quantile
    mahalanobis_df.loc[mahalanobis_df['mahalanobis_distance'] >= upper_quantile].to_csv(f"output/mahalanobis_distance_upper_quantile/{analysis_file}")
    plt.boxplot(mahalanobis_df['mahalanobis_distance'])
    plt.savefig(f"output/mahalanobis_boxplots/{analysis}.png")
    plt.clf()
pickle.dump(mahalanobis_threshold_dict, open("mahalanobis_thresholds.dict","wb"))

% Organic Matter (OM).csv
% Soil Nitrogen (N).csv
Carbon Analysis.csv
Complete Soil Analysis with Recommendations.csv
Complete Soil Analysis.csv
Exchangeable Acidity (Hp) Analysis.csv
Heavy Metals Analysis.csv
Olsen P Analysis.csv
Soil Bulk Density Analysis.csv


<Figure size 640x480 with 0 Axes>

In [61]:
mahalanobis_threshold_dict

{'% Organic Matter (OM)': 2.3648435901990266,
 '% Soil Nitrogen (N)': 2.341762560170896,
 'Carbon Analysis': 1.9769038128646055,
 'Complete Soil Analysis with Recommendations': 3.5182055578406537,
 'Complete Soil Analysis': 2.610094640001792,
 'Exchangeable Acidity (Hp) Analysis': 2.8418876183133563,
 'Heavy Metals Analysis': 2.041120041840065,
 'Olsen P Analysis': 5.1603069382258315,
 'Soil Bulk Density Analysis': 1.6971815684432907}